<a href="https://colab.research.google.com/github/R786P/data-science-roadmap-2025_2026/blob/main/17_cloud_mlops/01_gradio_deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pygit2==1.15.1
%cd /content
!git clone https://github.com/lllyasviel/Fooocus.git
%cd /content/Fooocus
!python entry_with_update.py --share --always-high-vram

In [1]:
# 17_cloud_mlops/01_gradio_deployment.ipynb
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import gradio as gr

# Load and prep data
df = pd.read_csv("https://raw.githubusercontent.com/IBM/telco-customer-churn-on-icp4d/master/data/Telco-Customer-Churn.csv")
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['TotalCharges'] = df['TotalCharges'].fillna(0)
df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0})

# Train model
X = df[['tenure', 'MonthlyCharges', 'TotalCharges']]
y = df['Churn']
model = RandomForestClassifier(n_estimators=50, random_state=42)
model.fit(X, y)

# Prediction function
def predict_churn(tenure, monthly_charges, total_charges):
    # Handle empty input
    if total_charges == 0 and tenure == 0:
        total_charges = monthly_charges  # reasonable default

    pred = model.predict([[tenure, monthly_charges, total_charges]])[0]
    prob = model.predict_proba([[tenure, monthly_charges, total_charges]])[0][1]
    result = "✅ Likely to Stay" if pred == 0 else "⚠️ Likely to Churn"
    confidence = f"Confidence: {prob:.2f}" if pred == 1 else f"Confidence: {1-prob:.2f}"
    return result, confidence

# Gradio UI
demo = gr.Interface(
    fn=predict_churn,
    inputs=[
        gr.Number(label="Tenure (months)", value=12),
        gr.Number(label="Monthly Charges ($)", value=70),
        gr.Number(label="Total Charges ($)", value=840)
    ],
    outputs=[
        gr.Text(label="Prediction"),
        gr.Text(label="Confidence")
    ],
    title="Customer Churn Predictor",
    description="Enter customer details to predict churn risk."
)

# Launch (will give public URL in Colab)
print("🚀 Launching Gradio app...")
demo.launch(share=True)

🚀 Launching Gradio app...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://725e4842b698144c07.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
